# Regular Expressions
## Optimization, Python, and Git

## Optimizing Regular Expressions
* Regular expressions are extremely powerful, but can be quite time consuming
* A Google search for "optimizing regular expressions" returns dozens of articles and blogs about the subject
* My basic rules of thumb:
    + Get it working first
    + Don't be afraid to look for another solution


## Why Regular Expressions can be Slow
* Some of this is implementation dependent, but regular expressions work by going through a string one character at a time, looking for matches
    * If there are a lot of comparisons to be made at each character in the string, this will slow the regex down
* Some regexes require backtracking to determine if there is a match or not
    * The more backtracking, the longer the regex will take to execute
    * A good resource on this is [Catastrophic Backtracking](http://www.regular-expressions.info/catastrophic.html)
        + Although written as an ad for their product, it does have a lot of helpful information

## Optimization Tip 1
* Don't use regular expressions if you don't have to
    * This is especially true if the pattern you are searching for is all literals
    

In [64]:
%%bash
time perl slow.pl

1


real	0m2.926s
user	0m2.928s
sys	0m0.000s


In [ ]:
# %load slow.pl
$string = "This is a string";
for (my $i=0; $i <= 10000000; $i++) {
	if($string =~ /string/){
		$found = 1;
	}
}
print $found;


In [62]:
%%bash
time perl fast.pl

1


real	0m2.785s
user	0m2.784s
sys	0m0.000s


In [ ]:
# %load fast.pl
$string = "This is a string";
for (my $i=0; $i <= 10000000; $i++) {
	if(index($string,"string") != -1){
		$found = 1;
	}
}
print $found;


## Optimization Tip 2
* If you can, use ^ and \$ anchors
    + Limiting where a match can occur can make a regex fail faster

In [69]:
%%bash
time perl anchored.pl


real	0m1.468s
user	0m1.464s
sys	0m0.000s


In [70]:
%%bash
time perl unanchored.pl


real	0m1.701s
user	0m1.700s
sys	0m0.000s


## Optimization Tip 3
* Avoid quantifiers if you don't need to use them
* If you need to use them, see if you can use the non-greedy version

In [74]:
%%bash
time perl greedy.pl

1


real	0m5.114s
user	0m5.112s
sys	0m0.000s


In [73]:
%%bash
time perl nongreedy.pl

1


real	0m5.203s
user	0m5.200s
sys	0m0.000s


## Optimization Tip 4
* Structure your alternations efficiently
    * Alternations are searched left-to-righ, so put the (suspected) most common first

In [75]:
%%bash
time perl good_alt.pl

1


real	0m6.460s
user	0m6.460s
sys	0m0.000s


In [76]:
%%bash
time perl bad_alt.pl

1


real	0m6.575s
user	0m6.572s
sys	0m0.000s


## Optimization Tip 5
* Use non-capturing groups
    * If you are just using grouping to apply a quantifier or something else over a part of a pattern, consider a non-capturing group `(?:pattern)`

In [77]:
%%bash
time perl capture.pl

1


real	0m4.056s
user	0m4.056s
sys	0m0.000s


In [78]:
%%bash
time perl noncapture.pl

1


real	0m3.833s
user	0m3.832s
sys	0m0.000s


# Regex in Python

## Intro to re Module
* Regular expressions are not built into the core python language
    * Available by importing the standard `re` module
* Matching and substitution are done using methods 
* To avoid having to escape the `\` character in Python so the regex can process use a raw string
    * `r'This is a raw python string\n'`

In [79]:
import re

## Simple Matching
* The `re` module has 4 methods to performing matching
    * re.match
    * re.search
    * re.findall
    * re.finditer
* All methods take the arguments (pattern, string, optional_flags)

In [80]:
if re.match(r'needle',r'Is there a needle in this haystack?'):
    print "match"

In [81]:
if re.search(r'needle',r'Is there a needle in this haystack?'):
    print "match"

match


## The Match Object
* Regular expressions don't evaluate to `true` or `false` in Python
    * If a match is found, a `MatchObject` is returned
    * If no match is found, `None` is returned
* The `MatchObject` can be used to access groups found in the match, as well as information such as position

In [82]:
re.search(r'needle',r'Is there a needle in this haystack?')

In [83]:
match = re.search(r'(\w+)\sneedle',r'Is there a needle in this haystack?')
print match.group(0)
print match.group(1)

a needle
a


## re.findall and re.finditer
* Rather than using a `g` modifier, Python has two specialized functions
* re.findall returns the groups themselves
* re.finditer returns an iterator over `MatchObjects` 

In [84]:
re.findall(r'\b\w*a\w*\b', r'Is there a needle in this haystack?')

['a', 'haystack']

In [85]:
re.findall(r'\b(\w*)a(\w*)\b', r'Is there a needle in this haystack?')

[('', ''), ('hayst', 'ck')]

## Backreferencing
* Backreferencing works exactly the same in Python
* Python also allows named groups, but personally I find it messy

In [86]:
re.findall(r'(\w)\1', r'Is there a needle in this haystack?')

['e']

In [87]:
re.findall(r'(?P<a_letter>\w)(?P=a_letter)', r'Is there a needle in this haystack?')

['e']

## Substitution
* Substitution is done using the `re.sub` method

```python
re.sub(pattern,replacement,string,count=0,flags=0)
```
* `re.sub` is global by default. To do only one substitution set the `count` parameter to 1
* `replacement` can be either a string or a function that takes a `MatchObject` as it's argument
* Back references are done using `\1` instead of `$1`

## Substitution Examples


In [88]:
re.sub(r'(\w)\1','oo',r'Is there a needle in this haystack?')

'Is there a noodle in this haystack?'

In [91]:
re.sub(r'(\w)\1',r'\1',r'Is there a needle in this haystack?')

'Is there a nedle in this haystack?'

## Splitting Strings
* The `re` module can split strings using the `split` method

```python
re.split(regex,string,limit,flags)
```

In [97]:
re.split(r'[aeiou]+',r'Is there a needle in this haystack?')

['Is th', 'r', ' ', ' n', 'dl', ' ', 'n th', 's h', 'yst', 'ck?']

## Using Flags
* Flags in Python are constans of the `re` module
    + re.I and re.IGNORECASE are equivalent to the i modifier in Perl
    + re.M and re.MULTILINE are equivalent to the m modifier in Perl
* To use multiple flags, you must "or" them together

## Flag Examples

In [98]:
re.search(r'n(\w)\1dle',r'Is there a\n NOODLE in this haystack')

In [102]:
match = re.search(r'n(\w)\1dle',r'Is there a\n NOODLE in this haystack',flags = re.I | re.M)
print match.start(), match.end(), match.pos, match.group(0), match.group(1)

 13 19 0 NOODLE O


## Compliling Regular Expressions
* If a regular expression is going to be used over and over again, you should compile it to the languages internal representation
    * Most languages have a concept of compilation
    * In Python, calling `re.compile(pattern,flags)` will return a `RegexObject`
* The methods of a `RegexObject` are mostly the same as the `re` module, but the pattern is no longer passsed as an argument

## Compiling Regular Expressions Examples

In [103]:
regex = re.compile(r'n(\w)\1dle',flags = re.I | re.M)
if regex.search('iS ThERe a \nNOoDLE iN This HaYStaCK'):
    print "Match"
    
print regex.sub(r"z\1\1",'Is there a noodle in Baltimore?')

for match in regex.finditer('You shouldn\'t sew your clothes with a noodle, no matter how many needles you have'):
    print match.group(0)

Match
Is there a zoo in Baltimore?
noodle
needle


# Intro to Git and GitHuB

## Git
* Git is a version control system widely used in open source software
    * Created by Linus Torvalds in 2005
* Is a distributed VCS
    * Everyone has a full copy of the repository, including history
* Git is pre-installed on most Linux systems, including GL
    * If you need to download it, follow the instructions at https://git-scm.com/downloads


## GitHub
* Even though Git is distributed, the files still need to be shared between collaborators
* Any publicly accessible web server could serve this purpose but
    * GitHub adds lot of nice features on top, including a nice web interface to view code in
* GitHub is not the only git host, but is by far the largest 
    * BitBucket is another popular one, gives unlimited free repositories
    

## Getting a Git repo
* If you are starting out with a new code base, simply
    + create a folder for your project
    + run `git init` inside the folder to make a git repository
* If you want to use existing code, say from GitHub, you can use the `git clone` command


In [104]:
%%bash
git clone https://github.com/bwilk7/433Fall17/ 

Cloning into '433Fall17'...


## Adding Changes to Git
* After modifying some of your code, it is a good idea to add the changes to the repository
* The command `git add FILE` adds a new version specific file to the repository
    + You can used wildcard to add many files at once, just be careful

## Commiting Changes
* After adding the files, the changes needed to be commited to repository
    * This calculates the difference between existing files and the ones youv'e added, as well as marks a specific point with a message
    
* To commit the added files in your Git repository use the command `git commit -m MESSAGE` where `MESSAGE` is some informative information about the change you made


## Pushing the Code
* If you want to push your code back to GitHub, that is make the repository on GitHub match your local repository, use the command `git push <remote> <branch>`
* If you cloned your code from GitHub, then GitHub will be set as the `origin`, so your command might be

```bash
git push origin master
```

* You can have multiple remote points, and multiple branches, but for this course, they shouldn't be needed

## Working Across Computers
* If you want to work on the same code across multile computers, thats fine!
* First clone the repository to the computers to want to work on
* After committing some changes and pushing them back to GitHub on computer A, you need to update computer B's repository
    * This is accomplished using the command `git pull origin`

##  Merges
* Git does it best to merge files together when multiple people have made changes on them
* Sometimes it is hard to figure out how to merge, and it will leave conflicts in your code
* To fix a conflict, edit the conflicted files, and add them again using `git add`
    + I don't expect conflicts to be an issue in this class